### The Rössler System

The **Rössler system** is a set of three coupled ordinary differential equations that describe a chaotic dynamical system. Introduced by Otto Rössler in 1976, it is widely studied as a simple model of chaos. The equations are:

\[
\begin{aligned}
    \frac{dx}{dt} &= -y - z, \\
    \frac{dy}{dt} &= x + ay, \\
    \frac{dz}{dt} &= b + z(x - c),
\end{aligned}
\]

where:
- \( x, y, z \) are the system's state variables,
- \( a, b, c \) are parameters that control the system's behavior.

---

#### Properties of the Rössler System

1. **Chaotic System**:
   - For certain parameter values, the system exhibits chaotic behavior characterized by sensitivity to initial conditions and the presence of a strange attractor.
   - Common parameter values for chaos are \( a = 0.2, b = 0.2, c = 5.7 \).

2. **Lyapunov Exponent**:
   - The largest **Lyapunov exponent** quantifies the rate at which nearby trajectories diverge, a hallmark of chaos. A positive Lyapunov exponent indicates chaos. ($\lambda_1$ =0.0714)

3. **Dissipative System**:
   - A system is **dissipative** if the volume of its state space contracts over time. For the Rössler system, this can be shown by computing the divergence of the vector field:
     $$
     \nabla \cdot \mathbf{F} = \frac{\partial f_x}{\partial x} + \frac{\partial f_y}{\partial y} + \frac{\partial f_z}{\partial z} = -a.
     $$
     Since \( a > 0 \), the system is dissipative, leading to the formation of an attractor.

4. **Applications**:
   - The Rössler system is used in modeling chaotic circuits, understanding turbulence, and exploring the fundamental properties of chaos.

---

#### Generating Time Series

Given an initial condition \( (x_0, y_0, z_0) \), the Rössler system can be integrated numerically to generate time series data. The output provides insight into the system's dynamics and serves as a benchmark for analyzing chaotic time series.


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def rossler_time_series(initial_condition, dt, num_iterations, params=(0.2, 0.2, 5.7)):
    """
    Generates a time series from the Rössler system.

    Args:
        initial_condition (tuple): Initial state (x0, y0, z0).
        dt (float): Time step for integration.
        num_iterations (int): Number of time steps.
        params (tuple): Parameters of the Rössler system (a, b, c).

    Returns:
        np.ndarray: Time series as a numpy array of shape (num_iterations, 3).
    """
    a, b, c = params

    def rossler_equations(t, state):
        x, y, z = state
        dx = -y - z
        dy = x + a * y
        dz = b + z * (x - c)
        return [dx, dy, dz]

    # Time span for integration
    t_span = (0, num_iterations * dt)
    t_eval = np.linspace(0, num_iterations * dt, num_iterations)

    # Solve the system using an ODE solver
    solution = solve_ivp(
        rossler_equations,
        t_span=t_span,
        y0=initial_condition,
        t_eval=t_eval,
        method='RK45'
    )

    return solution.y.T  # Transpose to shape (num_iterations, 3)

# Example usage
if __name__ == "__main__":
    # Initial condition and parameters
    initial_condition = (1.0, 1.0, 1.0)
    dt = 0.01
    num_iterations = 10000

    # Generate time series
    time_series = rossler_time_series(initial_condition, dt, num_iterations)

    # Plot the time series
    fig, ax = plt.subplots(3, 1, figsize=(10, 8))
    ax[0].plot(time_series[:, 0], label="x(t)", color="blue")
    ax[1].plot(time_series[:, 1], label="y(t)", color="orange")
    ax[2].plot(time_series[:, 2], label="z(t)", color="green")
    for a in ax:
        a.legend()
        a.set_xlabel("Time Step")
    plt.tight_layout()
    plt.show()

    # 3D Plot of the Rössler attractor
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(time_series[:, 0], time_series[:, 1], time_series[:, 2], lw=0.5)
    ax.set_title("Rössler Attractor")
    ax.set_xlabel("x(t)")
    ax.set_ylabel("y(t)")
    ax.set_zlabel("z(t)")
    plt.show()


### Student Exercise: Modeling the Rössler System

In this exercise, your task is to design, train, and evaluate a model capable of generating a time series resembling the dynamics of the **Rössler system**. Follow the steps outlined below to complete the assignment.

---

#### Tasks

1. **Generate the Dataset**:
   - Use the provided `rossler_time_series` function to create a dataset representing the Rössler system's dynamics.
   - Prepare the dataset for training a machine learning model by splitting it into input-output pairs suitable for sequence modeling.

2. **Train a Meta-Model**:
   - Develop a model (e.g., an RNN, LSTM, Transformer, or State-Space Model) that learns the underlying dynamics of the Rössler system.
   - Train the model to predict the next state \( x_{k+1}, y_{k+1}, z_{k+1} \) given the current and or previous states.

3. **Generate a New Time Series**:
   - Using the trained model, generate a new time series with:
     - \( dt = 0.01 \),
     - Initial condition: NumPy array of shape \( 1000 \times 3 \) with \( dt = 0.01 \).

In [ ]:
import numpy as np
import torch

def generate_time_series(initial_series, N, model_path):
    """
    Generates a time series of N iterations using a pre-trained model.

    Args:
        initial_series (np.ndarray): Initial series of shape (1000, 3).
        N (int): Number of iterations to generate.
        model_path (str): Path to the pre-trained model file.

    Returns:
        np.ndarray: Generated time series of shape (N, 3).
    """
    # Load the pre-trained model
    model = torch.load(model_path)
    model.eval()  # Set the model to evaluation mode

    # Initialize the time series with the first state
    generated_series = []
    current_state = initial_series #reshape/slice the initial_series as needed by your model

    # Generate the time series iteratively
    for _ in range(N):
        with torch.no_grad():
            # Predict the next state
            next_state = model(current_state)  # Assuming model outputs shape (1, 3)
        
        # Append to the generated series
        generated_series.append(next_state.squeeze(0).numpy())
        
        # Update the current state
        current_state = next_state

    return np.array(generated_series)


### Evaluation of the Exercise

To evaluate your submission, I will use the **`generate_time_series`** function with your trained model to generate a new time series. Starting from an initial condition, your model will be tasked with producing a time series of \( N \) iterations that reflects the dynamics of the Rössler system.

#### Evaluation Metrics

The generated time series will be assessed using the following metrics derived from dynamical systems theory:

1. **Lyapunov Exponent**:
   - I will compute the largest Lyapunov exponent from your generated time series and compare it with the theoretical value for the Rössler system.

2. **Correlation Dimension**:
   - The **correlation dimension** quantifies the fractal structure of the system's attractor. It is calculated based on the scaling of pairwise distances between points in the state space.
   - The correlation dimension provides insight into the complexity of the attractor and whether your model captures the chaotic dynamics of the Rössler system.